In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import pickle
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.svm import SVR
from sklearn import model_selection as ms
from scipy.stats import pearsonr
from sklearn.metrics import make_scorer
from sklearn import metrics
from sklearn.feature_selection import VarianceThreshold

rand_st=4

In [ ]:
output=pd.read_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx')
errors=pd.read_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx')

In [ ]:
elos_1=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\elos1.csv')
elos_2=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\elos2.csv')
elos_3=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\elos3.csv')
elos_4=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\elos4.csv')
features_1=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\features1.csv')
features_3=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\features3.csv')
features_2=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\features2.csv')
features_4=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\features4.csv')

In [ ]:
X_train_mean_1, X_test_mean_1, Y_train_mean_1, Y_test_mean_1 =ms.train_test_split(features_1, elos_1['MeanElos'], test_size = 0.3,random_state=rand_st)
X_train_diff_1, X_test_diff_1, Y_train_diff_1, Y_test_diff_1 =ms.train_test_split(features_1, elos_1['DiffElos'], test_size = 0.3,random_state=rand_st)

X_train_mean_2, X_test_mean_2, Y_train_mean_2, Y_test_mean_2 =ms.train_test_split(features_2, elos_2['MeanElos'], test_size = 0.3,random_state=rand_st)
X_train_diff_2, X_test_diff_2, Y_train_diff_2, Y_test_diff_2 =ms.train_test_split(features_2, elos_2['DiffElos'], test_size = 0.3,random_state=rand_st)

X_train_mean_3, X_test_mean_3, Y_train_mean_3, Y_test_mean_3 =ms.train_test_split(features_3, elos_3['MeanElos'], test_size = 0.3,random_state=rand_st)
X_train_diff_3, X_test_diff_3, Y_train_diff_3, Y_test_diff_3 =ms.train_test_split(features_3, elos_3['DiffElos'], test_size = 0.3,random_state=rand_st)

X_train_mean_4, X_test_mean_4, Y_train_mean_4, Y_test_mean_4 =ms.train_test_split(features_4, elos_4['MeanElos'], test_size = 0.3,random_state=rand_st)
X_train_diff_4, X_test_diff_4, Y_train_diff_4, Y_test_diff_4 =ms.train_test_split(features_4, elos_4['DiffElos'], test_size = 0.3,random_state=rand_st)

In [ ]:
model_mean=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
              kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

model_diff=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
          kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

svm_params =[{'kernel':['poly'],'degree':[2,3,4,5],'gamma':[0,1e-2, 1e-4,1e-6],'C':[0.001,0.01,0.1,1,10,100]},
            {'kernel':['rbf','sigmoid'],'gamma':[0,1e-2, 1e-4,1e-6],'C':[0.001,0.01,0.1,1,10,100]},
            {'kernel':['linear'],'C':[0.001,0.01,0.1,1,10,100]}]

In [ ]:
def find_mean_error(pred,elos):
    er=0
    count=0
    for ind, val in enumerate(pred):
        er+=abs(elos.iloc[ind,0]-val[0])
        er+=abs(elos.iloc[ind,1]-val[1])
        count+=2
    return round(er/count,3)

def find_abs_error(pred,elos):
    er=0
    for ind, val in enumerate(pred):
        er+=abs(elos.iloc[ind,0]-val[0])
        er+=abs(elos.iloc[ind,1]-val[1])
    return round(er,3)

def find_root_mean_squared_error(pred,elos):
    er=0
    count=0
    for ind, val in enumerate(pred):
        er+=(elos.iloc[ind,0]-val[0])**2
        er+=(elos.iloc[ind,1]-val[1])**2
        count+=2
    return round((er/count)**0.5,3)

In [ ]:
def predictions_report(pred_target,predictions):
    mae_report=mae(pred_target,predictions)
    print ('MAE = {:.3f}'.format(mae_report))
    
    rmse_report = (mse(pred_target, predictions))**0.5
    print ('RMSE = {:.3f}'.format(rmse_report))
    
    corr_coef = pearsonr(pred_target,predictions)
    print("Correlation coefficient = {:.3f}".format(corr_coef[0]))
    
    return (mae_report,rmse_report,corr_coef)

In [ ]:
def GridTune(params,classifiers,folds,X_train,Y_train):
    fscore = metrics.make_scorer(metrics.f1_score, average='weighted',greater_is_better=True)
    
    grid = ms.GridSearchCV(classifiers,
                           params,  # настройка параметров через кросс-валидацию
                           refit=True,    # переобучение по лучшим найденным параметрам   
                           scoring=fscore,
                           n_jobs=-1,
                           cv=ms.StratifiedKFold(n_splits=folds))
    
    grid_best_model = grid.fit(X_train,Y_train)

    print("Лучшие параметры для F-меры:")
    print(grid_best_model.best_params_)
    print('\nЛучший алгоритм:')
    print(grid_best_model.best_estimator_)
    return grid_best_model

### Подбор модели для 1 кластера

In [ ]:
start1 = time.time()

svr1=SVR(random_state=rand_st)
svm_model1=GridTune(svm_params,svr1,5,X_train_mean_1,Y_train_mean_1)
svm_pred1=svm_model1.predict(X_test_mean_1)
scores1=predictions_report(Y_test_mean_1,svr_pred1)
full_time1=round(time.time() - start1,3)
print(full_time1, 'c')

In [ ]:
output = output.append(pd.Series([str(model_mean),
                                 scores1[0],scores1[1], 
                                 scores1[2][0],full_time1,'Обучение на среднем ЭЛО по 1 кластеру'],
                                 index=output.columns ),
                      ignore_index=True)


output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)
output

In [ ]:
start2 = time.time()

svr2=model_diff.fit(X_train_diff_1,Y_train_diff_1)
svr_pred2=svr2.predict(X_test_mean_1)
scores2=predictions_report(Y_test_diff_1,svr_pred2)
full_time2=round(time.time() - start2,3)
print(full_time2, 'c')

In [ ]:
output = output.append(pd.Series([str(model_diff),
                                 scores2[0],scores2[1], 
                                 scores2[2][0],full_time2,'Обучение на разнице ЭЛО по 1 кластеру'],
                                 index=output.columns ),
                      ignore_index=True)


output

In [ ]:
output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)

In [ ]:
white_elos_1 = svr_pred1 + svr_pred2/2
black_elos_1 = svr_pred1-svr_pred2/2
pred_elos1=list(zip(white_elos_1,black_elos_1))
print(len(pred_elos1))
pred_elos1[:5]

In [ ]:
find_abs_error(pred_elos1,elos_1),find_mean_error(pred_elos1,elos_1),find_root_mean_squared_error(pred_elos1,elos_1)

In [ ]:
inner_mae_1=find_mean_error(pred_elos1,elos_1)

In [ ]:
errors =errors.append(pd.Series(['1 кластер','SVM',
                  find_abs_error(pred_elos1,elos_1),
                  find_mean_error(pred_elos1,elos_1),
                   find_root_mean_squared_error(pred_elos1,elos_1) ],
                                   index=errors.columns ),
                      ignore_index=True)

errors.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx',index=False)
errors

In [ ]:
errors.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx',index=False)

### Подбор модели для 2 кластера

In [ ]:
start3 = time.time()

svr3=model_mean.fit(X_train_mean_2,Y_train_mean_2)
svr_pred3=svr3.predict(X_test_mean_2)
scores3=predictions_report(Y_test_mean_2,svr_pred3)
full_time3=round(time.time() - start3,3)
print(full_time3, 'c')

In [ ]:
output = output.append(pd.Series([str(model_mean),
                                 scores3[0],scores3[1], 
                                 scores3[2][0],full_time3,'Обучение на среднем ЭЛО по 2 кластеру'],
                                 index=output.columns ),
                      ignore_index=True)


output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)
output

In [ ]:
start4 = time.time()

svr4=model_diff.fit(X_train_diff_2,Y_train_diff_2)
svr_pred4=svr4.predict(X_test_mean_2)
scores4=predictions_report(Y_test_diff_2,svr_pred4)
full_time4=round(time.time() - start4,3)
print(full_time4, 'c')

In [ ]:
output = output.append(pd.Series([str(model_diff),
                                 scores4[0],scores4[1], 
                                 scores4[2][0],full_time4,'Обучение на разнице ЭЛО по 2 кластеру'],
                                 index=output.columns ),
                      ignore_index=True)


output

In [ ]:
output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)

In [ ]:
white_elos_2 = svr_pred3 + svr_pred3/2
black_elos_2 = svr_pred3-svr_pred4/2
pred_elos2=list(zip(white_elos_2,black_elos_2))
print(len(pred_elos2))
pred_elos2[:5]

In [ ]:
find_abs_error(pred_elos2,elos_2),find_mean_error(pred_elos2,elos_2),find_root_mean_squared_error(pred_elos2,elos_2)

In [ ]:
inner_mae_2=find_mean_error(pred_elos2,elos_2)

In [ ]:
errors =errors.append(pd.Series(['2 кластер','SVM',
                        find_abs_error(pred_elos2,elos_2),
                        find_mean_error(pred_elos2,elos_2),
                        find_root_mean_squared_error(pred_elos2,elos_2)],
                                index=errors.columns ),
                      ignore_index=True)

errors.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx',index=False)
errors

### Подбор модели для 3 кластера

In [ ]:
start5 = time.time()

svr5=model_mean.fit(X_train_mean_3,Y_train_mean_3)
svr_pred5=svr5.predict(X_test_mean_3)
scores5=predictions_report(Y_test_mean_3,svr_pred5)
full_time5=round(time.time() - start5,3)
print(full_time5, 'c')

In [ ]:
output = output.append(pd.Series([str(model_mean),
                                 scores5[0],scores5[1], 
                                 scores5[2][0],full_time5,'Обучение на среднем ЭЛО по 3 кластеру'],
                                 index=output.columns ),
                      ignore_index=True)


output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)
output

In [ ]:
start6 = time.time()

svr6=model_diff.fit(X_train_diff_3,Y_train_diff_3)
svr_pred6=svr6.predict(X_test_mean_3)
scores6=predictions_report(Y_test_diff_3,svr_pred6)
full_time6=round(time.time() - start6,3)
print(full_time6, 'c')

In [ ]:
output = output.append(pd.Series([str(model_diff),
                                 scores6[0],scores6[1], 
                                 scores6[2][0],full_time6,'Обучение на разнице ЭЛО по 3 кластеру'],
                                 index=output.columns ),
                      ignore_index=True)


output

In [ ]:
output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)

In [ ]:
white_elos_3 = svr_pred5 + svr_pred6/2
black_elos_3 = svr_pred5-svr_pred6/2
pred_elos3=list(zip(white_elos_3,black_elos_3))
print(len(pred_elos3))
pred_elos3[:5]

In [ ]:
find_abs_error(pred_elos3,elos_3),find_mean_error(pred_elos3,elos_3),find_root_mean_squared_error(pred_elos3,elos_3)

In [ ]:
inner_mae_3=find_mean_error(pred_elos3,elos_3)

In [ ]:
errors =errors.append(pd.Series(['3 кластер','SVM',
                                 find_abs_error(pred_elos3,elos_3),
                                 find_mean_error(pred_elos3,elos_3),
                                 find_root_mean_squared_error(pred_elos3,elos_3)],
                                index=errors.columns ),
                      ignore_index=True)

errors.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx',index=False)
errors

### Подбор модели для 4 кластера

In [ ]:
start7 = time.time()

svr7=model_mean.fit(X_train_mean_4,Y_train_mean_4)
svr_pred7=svr7.predict(X_test_mean_4)
scores7=predictions_report(Y_test_mean_4,svr_pred7)
full_time7=round(time.time() - start7,3)
print(full_time7, 'c')

In [ ]:
output = output.append(pd.Series([str(model_mean),
                                 scores7[0],scores7[1], 
                                 scores7[2][0],full_time7,'Обучение на среднем ЭЛО по 4 кластеру'],
                                 index=output.columns ),
                      ignore_index=True)


output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)
output

In [ ]:
start8 = time.time()

svr8=model_diff.fit(X_train_diff_4,Y_train_diff_4)
svr_pred8=svr8.predict(X_test_mean_4)
scores8=predictions_report(Y_test_diff_4,svr_pred8)
full_time8=round(time.time() - start8,3)
print(full_time8, 'c')

In [ ]:
output = output.append(pd.Series([str(model_diff),
                                 scores8[0],scores8[1], 
                                 scores8[2][0],full_time8,'Обучение на разнице ЭЛО по 4 кластеру'],
                                 index=output.columns ),
                      ignore_index=True)


output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)
output

In [ ]:
white_elos_4 = svr_pred7 + svr_pred8/2
black_elos_4 = svr_pred7-svr_pred8/2
pred_elos4=list(zip(white_elos_4,black_elos_4))
print(len(pred_elos4))
pred_elos4[:5]

In [ ]:
find_abs_error(pred_elos4,elos_4),find_mean_error(pred_elos4,elos_4),find_root_mean_squared_error(pred_elos4,elos_4)

In [ ]:
inner_mae_4=find_mean_error(pred_elos4,elos_4)

In [ ]:
errors =errors.append(pd.Series(['4 кластер','SVM',find_abs_error(pred_elos4,elos_4),
                                 find_mean_error(pred_elos4,elos_4),
                                 find_root_mean_squared_error(pred_elos4,elos_4)],
                                index=errors.columns ),
                      ignore_index=True)

errors.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx',index=False)
errors

In [ ]:
svr_mae=(inner_mae_4+inner_mae_3+inner_mae_2+inner_mae_1)/4
svr_mae

In [ ]:
svr_abs=(find_abs_error(pred_elos1,elos_1)+
        find_abs_error(pred_elos2,elos_2)+
        find_abs_error(pred_elos3,elos_3)+
        find_abs_error(pred_elos4,elos_4))/4
svr_abs

In [ ]:
svr_rmse=(find_root_mean_squared_error(pred_elos1,elos_1)+
         find_root_mean_squared_error(pred_elos2,elos_2)+
         find_root_mean_squared_error(pred_elos3,elos_3)+
         find_root_mean_squared_error(pred_elos4,elos_4))/4
svr_rmse

In [ ]:
errors = errors.append(pd.Series(['Общая ошибка','SVM',
                  svr_abs,
                  svr_mae,
                 svr_rmse],index=errors.columns) ,ignore_index=True)

errors.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx',index=False)
errors